# Let's Build Personalized Website using Agentic Workflows and Inferentia

Notes: 
1. Use the native Jupyter Notebook to leverage the full features of this notebook
2. Ensure you have Bedrock connection with access to Titan Models 

## 1. Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import base64
import boto3
import io
import os
import json
import re
import importlib
import shutil
import re
import numpy as np
import matplotlib.pyplot as plt

from src import Bedrock

from ipywidgets import widgets
from IPython.core.display import display, HTML
from PIL import Image


In [ ]:
importlib.reload(Bedrock)

### Useful functions

In [ ]:
# fetch context from the response
def get_contexts(retrievalResults):
    total_text = ""
    for result in retrievalResults.get("retrievalResults", []):
        content = result.get("content", {})
        text = content.get("text", "")
        # Remove the metadataAttributes content from the text
        text = re.sub(r'\{.*?metadataAttributes.*?\}', '', text)
        total_text += text
    return total_text

In [ ]:
def build_profile(UserProfile):
    profile = f"Your customer is {UserProfile['Name']}, located in {UserProfile['Location']}. They are a {UserProfile['Industry']} company. Their mission statement is `{UserProfile['Mission']}."
    return profile

In [ ]:
def get_labels_and_desc(pattern, text):  
    matches = re.findall(pattern, text)
    myList = [] #(label, description)
    for match in matches:
        title = match[0]
        description = match[1]
        myList.append((title,description))
    return myList

In [ ]:
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = io.BytesIO(base64_image)
  return Image.open(buffer)
 
# Display PIL images as grid
def display_image(image=None,width=500,height=500):
    img = image.resize((width, height))
    display(img)

### Declerations

In [ ]:
# create general model inputs 
# do not edit any content in this section

#image_modelId = 'amazon.titan-image-generator-v1'
#model_id = {'Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
#           'Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
#           'StableDiff': 'stability.stable-diffusion-xl-v1'}

In [ ]:
# ENV VARIABLES
kbId = '2TBRS1XZBR'
model_llama3p1_endpoint = 'nous-research-Meta-Llama-3-8B-2024-09-23-19-42-59-162-endpoint'
model_stablediffV1_endpoint = 'huggingface-pytorch-inference-neuronx-2024-10-01-23-05-27-894'

### Checking the endpoint status

In [ ]:
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_llama3p1_endpoint)
print(f"Status for {model_llama3p1_endpoint}: {resp['EndpointStatus']}")

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_stablediffV1_endpoint)
print(f"Status for {model_stablediffV1_endpoint}: {resp['EndpointStatus']}")

### INPUT

Offerings

In [ ]:
with open("./references/offerings.json", "r") as file:
    offerings = json.load(file)

User Profiles

In [ ]:
Profiles = {
    'Construction-Example': {
        'Name': 'Example Corp Construction Inc',
        'Industry': 'Construction',
        'Location': 'New York City, NY',
        'Mission': 'Building a sustainable future for New York'
    },
    'Manufacturing-Example': {
        'Name': 'Example Corp Manuf LLC',
        'Industry': 'Manufacturing',
        'Location': 'San Jose, CA',
        'Mission': 'Building the next generation Electric Vehicles'
    },
    'Mining-Example': {
        'Name': 'Example Corp Mining Inc',
        'Industry': 'Mining',
        'Location': 'Bisbee, AZ',
        'Mission': 'Extracting the value for America'
    }
}

Now a user signs in (ITERATION)

In [ ]:
# ITERATION - SELECTION:
selected_profile = 'Manufacturing-Example'

In [ ]:
UserProfile= Profiles[selected_profile]
offering = re.sub(r'\n#?', ' ', offerings[customer['Industry']])
project = selected_profile + "_Website_Test"

### Runtime

#### 1) Natural Language description of customer profile

In [ ]:
customer = UserProfile.copy()
customer['Description'] = build_profile(UserProfile)
print(customer['Description'])

#### 2) Retreieve painpoints of customer industry / any context
This flow uses RAG to retreive documents

In [ ]:
query = f"{customer['Description']}\nList the industry pain-points and challenges"
print(f"Here is your customer:\n{query}")

In [ ]:
# Let's find the context for the painpoints and challenges for this customer using RAG.
context_painpoints = Bedrock.dec_retrieve(query, kbId, numberOfResults=5)
contexts_painpoints = get_contexts(context_painpoints)
contexts_painpoints = re.sub(r'\n#?', ' ', contexts_painpoints)
print(f"These are the applicable pain-points/challenges:\n{contexts_painpoints}")

### 3) Prompting AI to describe a website personalized for this profile and background
Here the prompting is critical. Please see the guidance provided to the LLM. We also ask the response to be in two parts; 1/ Description, 2/ Visual Elements. The second part is needed for an easier traffic move to image generation or finding image.

#### 3.1) Finding the possible painpoints using LLM

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Research Document ***
{contexts_painpoints}
*** End of Research Document ***


Question: {customer['Description']} Based on the above research, list top-3 challenges and explain we understand their challanges. Do not use any company name.
Provide your response in bullet points without any preamble (e.g. • a pain-point: one sentence description, • pain-point: one sentence description, etc.). 

"""
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
%%time
response_model= smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_painpoints = json.load(response_model['Body'])
Top_pain_points_text= response_painpoints['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top painpoints:\n{Top_pain_points_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'
Top_painpoints = get_labels_and_desc(pattern, Top_pain_points_text)
[print(f"{key}: {value}\n")for (key, value) in Top_painpoints]

#### 3.2) Finding the personalized offerings using LLM

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Offerings Document ***
{offering}
*** Offerings Document ***


Question: {customer['Description']} Their top-3 painpoints are {Top_pain_points}.

List and briefly advertise the top-3 offerings for {customer['Name']} based on the offerings document above.
Provide your list in bullet points without any preamble (e.g. • an offering: short description, • an offering: short description, ...). 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
params = {"max_new_tokens": 384, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_offerings = json.load(response_model['Body'])
Top_offerings_text= response_offerings['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top offerings:\n{Top_offerings_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'
Top_offerings = get_labels_and_desc(pattern, Top_offerings_text)
[print(f"{key}: {value}\n")for (key, value) in Top_offerings]

#### 3.3) Art Design

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a UI/UX designer personalizing visual website content such as hero image and icons tailored to customer profile, personalized offerings and pain-points. 
Your task is to describe a hero image and six icons. For the Hero Image, design an attention-grabbing image that relates to the customer's industry, location, and mission. For icons, design simple, memorable icons that visually represent the concepts.

Customer Profile:
- Industry: {customer['Industry']}
- Location: {customer['Location']}
- Company Motto/Mission: {customer['Mission']}

For the Hero Image:
- Describe a scene that represents the customer's industry and mission
- Include elements that reflect the customer location
- Mention 1-2 key visual elements

For each icon:
- Describe a simple, easily recognizable shape or symbol
- Mention one key visual element
- Call our that this is an 'icon'.

Each icon should be distinct yet part of a cohesive set in style and color.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Provide your response in the following structured format:

** Hero Image[Description in 1-2 sentences]**
** {Top_painpoints[0][0]} Icon [Description in one sentence]**
** {Top_painpoints[1][0]} Icon [Description in one sentence]**
** {Top_painpoints[2][0]} Icon [Description in one sentence]**
** {Top_offerings[0][0]} Icon [Description in one sentence]**
** {Top_offerings[1][0]} Icon [Description in one sentence]**
** {Top_offerings[2][0]} Icon [Description in one sentence]**

Provide your response using the exact format above, without any preamble.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_art= json.load(response_model['Body'])
art_descriptions_text = response_art['generated_text'].split('<|eot_id|>')[0]\
                        .replace('\n',', ').replace(', ,', '').replace('**,','**')
print(f"Raw Output - top offerings:\n{art_descriptions_text}\n")

In [ ]:
# Creating the variable for the art descriptions that will help us to create a better prompt later
art_descriptions = art_descriptions_text.split("**")[1:]
art_descriptions = {art_descriptions[i].strip():{"Description":art_descriptions[i+1].strip(), \
                    'FileName':art_descriptions[i].replace(' ','')+'.jpg'} for i in range(0,int(len(art_descriptions)),2)}

In [ ]:
print(f"Here is the art and their descriptions:\n {json.dumps(art_descriptions, indent=4)}")

In [ ]:
response_painpoints = json.load(response_model['Body'])
Top_pain_points_text= response_painpoints['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top painpoints:\n{Top_pain_points_text}\n")

In [ ]:
offering_section = f"""
  <section class="offerings">
    <h2>Our Offerings</h2>
    <div class="offering-cards">
      <div class="offering-card">
        <img src="{art_descriptions[Top_offerings[0][0]+' Icon']['FileName']}" alt="{Top_offerings[0][0]}">
        <h3>{Top_offerings[0][0]}</h3>
        <p>{Top_offerings[0][1]}</p>
      </div>
      <div class="offering-card">
        <img src="{art_descriptions[Top_offerings[1][0]+' Icon']['FileName']}" alt="{Top_offerings[1][0]}">
        <h3>{Top_offerings[1][0]}</h3>
        <p>{Top_offerings[1][1]}</p>
      </div>
      <div class="offering-card">
        <img src="{art_descriptions[Top_offerings[2][0]+' Icon']['FileName']}" alt="{Top_offerings[2][0]}">
        <h3>{Top_offerings[2][0]}</h3>
        <p>{Top_offerings[2][1]}</p>
      </div>
    </div>
  </section>
"""

In [ ]:
painpoint_section = f"""
  <section class="pain-points">
    <h2>Observing {customer['Industry']} Industry Challenges</h2>
    <div class="pain-point-cards">
      <div class="pain-point-card">
        <img src="{art_descriptions[Top_painpoints[0][0]+' Icon']['FileName']}" alt="{Top_painpoints[0][0]}">
        <h3>{Top_painpoints[0][0]}</h3>
        <p>{Top_painpoints[0][1]}</p>
      </div>
      <div class="pain-point-card">
        <img src="{art_descriptions[Top_painpoints[1][0]+' Icon']['FileName']}" alt="{Top_painpoints[1][0]}">
        <h3>{Top_painpoints[1][0]}</h3>
        <p>{Top_painpoints[1][1]}</p>
      </div>
      <div class="pain-point-card">
        <img src="{art_descriptions[Top_painpoints[2][0]+' Icon']['FileName']}" alt="{Top_painpoints[2][0]}">
        <h3>{Top_painpoints[2][0]}</h3>
        <p>{Top_painpoints[2][1]}</p>
      </div>
    </div>
  </section>
"""

In [ ]:
full_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>AnyCompany Consulting Personalized Page</title>
  <link rel="stylesheet" href="styles.css">
  <script src="script.js" defer></script>
</head>
<body>
  <header>
    <nav>
      <div class="logo">
        <img src="anycompany_logo.jpg" alt="AnyCompany Consulting Logo">
      </div>
      <ul class="menu">
        <li><a href="#">Home</a></li>
        <li><a href="#">About</a></li>
        <li><a href="#">Services</a></li>
        <li><a href="#">Contact</a></li>
      </ul>
    </nav>
  </header>

  <section class="hero">
    <div class="hero-content">
      <h1>{{CREATE A PERSONALIZED HERO TITLE}}</h1>
      <p>{{CREATE A ENGAGING TEXT TO PIQUE THE ATTENTION}}</p>
    </div>
  </section>

{offering_section}

{painpoint_section}

  <section class="testimonials">
    <h2>What Our Clients Say</h2>
    <div class="testimonial-slider">
      <div class="testimonial-slide">
        <blockquote>
          <p>"AnyCompany Consulting's expert consulting services have been invaluable in streamlining our operations and optimizing our construction processes, resulting in increased efficiency and cost savings"</p>
          <cite>- John Smith, CTO from Example Corp Solutions</cite>
        </blockquote>
      </div>
      <div class="testimonial-slide">
        <blockquote>
          <p>"AnyCompany Consulting's top-notch consulting solutions have transformed our organization, enabling us to tackle complex challenges, fostering innovation, and propelling our growth to new heights."</p>
          <cite>- Alejandro Rosalez, COO of AnyDepartment Stores</cite>
        </blockquote>
      </div>
      <div class="testimonial-slide">
        <blockquote>
          <p>"AnyCompany Consulting's strategic advisory services have been instrumental in strengthening our governance frameworks, enhancing transparency, and ensuring compliance with regulatory requirements, facilitating seamless public service delivery."</p>
          <cite>- Jane Doe, Governor of AnyGovernment</cite>
        </blockquote>
      </div>
    </div>
  </section>

  <section class="cta">
    <h2>{{CREATE A SHORT CATCHING CALL TO ACTION LABEL}}</h2>
    <p>{{BRIEFLY EXPLAIN WHY THEY SHOULD MOVE WITH THE CALL TO ACTION}}</p>
    <a href="#" class="cta-button">Get Started</a>
  </section>

  <footer>
    <div class="footer-content">
      <div class="company-info">
        <h3>AnyCompany Consulting</h3>
        <p>123 Main Street<br>New York, NY 10001</p>
        <p>Phone: (123) 456-7890</p>
        <p>Email: info@anycompanyconsulting.com</p>
      </div>
      <div class="social-links">
        <h3>Follow Us</h3>
        <ul>
          <li><a href="#"><img src="facebook-icon.png" alt="Facebook"></a></li>
          <li><a href="#"><img src="twitter-icon.png" alt="Twitter"></a></li>
          <li><a href="#"><img src="linkedin-icon.png" alt="LinkedIn"></a></li>
        </ul>
      </div>
    </div>
    <p class="copyright">&copy; 2023 AnyCompany Consulting. All rights reserved.</p>
  </footer>

  <script>
    // Testimonial Slider
    const testimonialSlides = document.querySelectorAll('.testimonial-slide');
    let currentSlide = 0;

    function showSlide(n) {
      testimonialSlides.forEach((slide, index) => {
        slide.style.display = index === n ? 'block' : 'none';
      });
    }

    function nextSlide() {
      currentSlide = (currentSlide + 1) % testimonialSlides.length;
      showSlide(currentSlide);
    }

    showSlide(currentSlide);
    setInterval(nextSlide, 5000);
  </script>
</body>
</html>

/* CSS */
@import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');

body {
  font-family: 'Roboto', sans-serif;
  margin: 0;
  padding: 0;
  color: #333;
}

header {
  background-color: #f2f2f2;
  padding: 20px;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

nav {
  display: flex;
  justify-content: space-between;
  align-items: center;
  max-width: 1200px;
  margin: 0 auto;
}

.logo img {
  max-height: 40px;
}

.menu {
  list-style: none;
  margin: 0;
  padding: 0;
  display: flex;
}

.menu li {
  margin-left: 20px;
}

.menu a {
  text-decoration: none;
  color: #333;
  font-weight: bold;
  transition: color 0.3s ease;
}

.menu a:hover {
  color: #ff6600;
}

.hero {
  background-image: url('{art_descriptions['Hero Image']['FileName'}');
  background-size: cover;
  background-position: center;
  height: 500px;
  display: flex;
  justify-content: center;
  align-items: center;
  text-align: center;
  color: #fff;
}

.hero-content {
  max-width: 800px;
  padding: 20px;
  background-color: rgba(0, 0, 0, 0.6);
  border-radius: 10px;
}

.hero-content h1 {
  font-size: 36px;
  margin-bottom: 20px;
}

.hero-content p {
  font-size: 18px;
  margin-bottom: 30px;
}

section {
  padding: 60px 20px;
  max-width: 1200px;
  margin: 0 auto;
}

section h2 {
  text-align: center;
  margin-bottom: 40px;
  font-size: 32px;
}

.offering-cards,
.pain-point-cards {
  display: flex;
  justify-content: space-between;
  flex-wrap: wrap;
}

.offering-card,
.pain-point-card {
  background-color: #f2f2f2;
  padding: 20px;
  border-radius: 10px;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
  text-align: center;
  max-width: 300px;
  margin-bottom: 30px;
  transition: transform 0.3s ease;
}

.offering-card:hover,
.pain-point-card:hover {
  transform: translateY(-5px);
}

.offering-card img,
.pain-point-card img {
  max-width: 80px;
  margin-bottom: 20px;
}

.offering-card h3,
.pain-point-card h3 {
  font-size: 20px;
  margin-bottom: 10px;
}

.offering-card p,
.pain-point-card p {
  font-size: 16px;
  color: #666;
}

.testimonials {
  background-color: #f2f2f2;
  padding: 60px 20px;
}

.testimonial-slider {
  max-width: 800px;
  margin: 0 auto;
  position: relative;
}

.testimonial-slide {
  display: none;
  text-align: center;
  padding: 40px;
  background-color: #fff;
  border-radius: 10px;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

.testimonial-slide blockquote {
  font-size: 18px;
  line-height: 1.6;
  margin-bottom: 20px;
}

.testimonial-slide cite {
  font-style: normal;
  font-weight: bold;
}

.cta {
  background-color: #ff6600;
  color: #fff;
  padding: 60px 20px;
  text-align: center;
}

.cta h2 {
  font-size: 36px;
  margin-bottom: 20px;
}

.cta p {
  font-size: 18px;
  margin-bottom: 30px;
}

.cta-button {
  display: inline-block;
  background-color: #fff;
  color: #ff6600;
  text-decoration: none;
  padding: 12px 24px;
  border-radius: 5px;
  font-weight: bold;
  transition: background-color 0.3s ease, color 0.3s ease;
}

.cta-button:hover {
  background-color: #333;
  color: #fff;
}

footer {
  background-color: #333;
  color: #fff;
  padding: 40px 20px;
}

.footer-content {
  display: flex;
  justify-content: space-between;
  max-width: 1200px;
  margin: 0 auto;
}

.company-info h3,
.social-links h3 {
  font-size: 20px;
  margin-bottom: 10px;
}

.company-info p {
  margin-bottom: 5px;
}

.social-links ul {
  list-style: none;
  margin: 0;
  padding: 0;
  display: flex;
}

.social-links li {
  margin-right: 10px;
}

.social-links a {
  display: block;
  width: 30px;
  height: 30px;
  transition: transform 0.3s ease;
}

.social-links a:hover {
  transform: scale(1.2);
}

.social-links img {
  max-width: 100%;
  height: auto;
}

.copyright {
  text-align: center;
  margin-top: 20px;
  font-size: 14px;
}

/* Responsive Styles */
@media (max-width: 768px) {
  .offering-cards,
  .pain-point-cards {
    flex-direction: column;
    align-items: center;
  }

  .offering-card,
  .pain-point-card {
    max-width: 100%;
  }

  .footer-content {
    flex-direction: column;
    text-align: center;
  }

  .social-links {
    margin-top: 20px;
  }
}
"""

#### 5) Image Creation based on Description


##### 5.1) Creating the imagery from the descriptions
Notice that if the image is "icon" then 512 x 512 otherwise 1024 x 1024

In [ ]:
%%time
# WE HAVE TO OPTIMIZE THIS FOR ICONS.
from sagemaker.huggingface.model import HuggingFacePredictor
predictor_SDv1 = HuggingFacePredictor(model_stablediffV1_endpoint)
params = {
    "num_inference_steps" : 25,
    "negative_prompt" : "disfigured, ugly, deformed, unprofessional"
    }
images = []
image_files = []
for art, content in art_descriptions.items():
    if 'Icon' in art:
        art_height = 512
        art_width = 512
    else:
        art_height = 1024
        art_width = 1024
    
    prompt = content['Description']
    response = predictor_SDv1.predict(data={"inputs": prompt,"parameters":params})

    image = decode_base64_image(response["generated_images"][0])
    image_files.append(content['FileName'])

    # Save the image
    image.save(f"{content['FileName']}")
    images.append(np.array(image))

In [ ]:
print("Number of visual assets created: {}".format(len(image_files)))

##### b) Invoke Stable Diffusion
Here we assumed that the image description is the prompt for Text-to-Image. We successively call the APIs and then save the files.

In [ ]:
shutil.copy("./references/anycompany_logo.jpg", "./anycompany_logo.jpg")

Let's plot the images generated

In [ ]:
# Plot the images
fig, axs = plt.subplots(1, len(images), figsize=(30, 20))

for i, (image, ax, filename) in enumerate(zip(images, axs, image_files)):
    ax.imshow(image)
    ax.set_title(filename, fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

#### 6) Let's Generate our HTML file

First, we need to extract the response for the Section 1: Detailed Description from *response_personalized_website*

In [ ]:
# Extract Section-1 using regular expressions
#section_1_pattern = r"Section 1: Detailed Website Description\n\n(.*?)\n\nSection 2:"
#match = re.search(section_1_pattern, response_personalized_website, re.DOTALL)

#if match:
#    response_detailed_explanation = match.group(1)
#    print(f"Here is the Section-1 extracted from the LLM response:\n\n{response_detailed_explanation}")
#else:
#    print("Section-1 not found in the input text.")

Here we will use Haiku to create the HTML file.

In [ ]:
prompt = f"""
You are an experienced front-end web developer specializing in creating accessible, responsive, and visually appealing websites. Your task is to generate the complete HTML, CSS, and JavaScript code that accurately implements the provided 'Website Description' while adhering to the specified guidelines.

<website description>
Know that, this your Design Guideline (Requirements):
{design_guideline}

You use the testimonials as follows;
{testimonials}

Website Description:
{response_personalized_website}
</website description>

Please carefully read the 'Website Description' line by line, and then generate the HTML, CSS, and JavaScript code required to build the described website while following the specified design guidelines and requirements.

Provide the HTML, CSS, and JavaScript code directly, starting with the <!DOCTYPE html> declaration, without any preamble or introduction.

"""

In [ ]:
%%time
response_html = Bedrock.invoke(prompt=prompt, modelID=model_id['Haiku'], max_tokens = 4096, temp = 0)

In [ ]:
#print(response_html)

#### Putting the assets into a single folder

In [ ]:
#Creating the folder for our project
try:
    os.mkdir(project)
    print(f"Project Folder: {project}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from IPython.display import HTML

In [ ]:
with open('main.html', 'w', encoding='utf-8') as file:
    file.write(response_html)

In [ ]:
project_files = list(visual_assets.keys())
project_files.append('main.html')

In [ ]:
for file in project_files:
    destination = project + "/" + file
    shutil.move(file, destination)